In [25]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

API_KEY = os.getenv("API_KEY")
URL = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000001&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"
response = requests.get(URL)
result= response.json()
result.keys() 


dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [26]:
len(result["Trip"])

9

In [27]:
result["Trip"][0].keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum'])

In [28]:
result["Trip"][0]["LegList"]["Leg"][0]["Origin"]["name"]


'Stockholm Centralstation'

In [29]:
result["Trip"][0]["LegList"]["Leg"][0]["Destination"]["name"]   

'Malmö Centralstation'

In [30]:
def get_trips(origin_id=740000001, destination_id=740098001):
    """origing_id and destination_id can be found from Stop lookup API"""
    url = f"https://api.resrobot.se/v2.1/trip?format=json&originId={origin_id}&destId={destination_id}&passlist=true&showPassingPoints=true&accessId={API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")


result = get_trips()
result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [36]:
# checks first trip keys
example_trip = result["Trip"][0]
example_trip.keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum', 'transferCount'])

In [39]:
example_trip['Origin']

{'name': 'Stockholm Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
 'extId': '740000001',
 'lon': 18.058151,
 'lat': 59.330136,
 'routeIdx': 0,
 'prognosisType': 'PROGNOSED',
 'time': '12:41:00',
 'date': '2025-01-07',
 'minimumChangeDuration': 'PT20M'}

In [40]:
example_trip['Destination']

{'name': 'Göteborg Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
 'extId': '740000002',
 'lon': 11.973479,
 'lat': 57.708895,
 'routeIdx': 18,
 'prognosisType': 'PROGNOSED',
 'time': '17:00:00',
 'date': '2025-01-07',
 'minimumChangeDuration': 'PT15M'}

In [43]:
example_trip['LegList']['Leg']

[{'Origin': {'name': 'Stockholm Centralstation',
   'type': 'ST',
   'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
   'extId': '740000001',
   'lon': 18.058151,
   'lat': 59.330136,
   'routeIdx': 0,
   'prognosisType': 'PROGNOSED',
   'time': '12:41:00',
   'date': '2025-01-07',
   'minimumChangeDuration': 'PT20M'},
  'Destination': {'name': 'Hallsberg station',
   'type': 'ST',
   'id': 'A=1@O=Hallsberg station@X=15110391@Y=59066698@U=1@L=740000077@',
   'extId': '740000077',
   'lon': 15.110391,
   'lat': 59.066698,
   'routeIdx': 3,
   'prognosisType': 'PROGNOSED',
   'time': '14:19:00',
   'date': '2025-01-07',
   'minimumChangeDuration': 'PT10M'},
  'Notes': {'Note': [{'value': 'Obligatorisk platsbokning',
     'key': 'AF',
     'type': 'A',
     'routeIdxFrom': 0,
     'routeIdxTo': 3,
     'txtN': 'Obligatorisk platsbokning'},
    {'value': 'Rullstolslyft',
     'key': 'AN',
     'type': 'A',
     'routeIdxFrom': 0,
     'routeIdxTo': 3,
     't